# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
# %%bash 
# pip install duckdb
# pip install jupysql
# pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters=True

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
SELECT
    'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [4]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS
SELECT
    *
FROM read_csv_auto('../../datasets/raw/competencia_01_crudo.csv')

,Count
0,491063


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [5]:
%%sql
SELECT
    *
FROM competencia_01
LIMIT 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [6]:
%%sql
SELECT
    foto_mes
    , COUNT(*) AS cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
FROM competencia_01
GROUP BY foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [7]:
%%sql
SELECT
    numero_de_cliente,
    COUNT(*) AS cantidad
FROM competencia_01
GROUP BY numero_de_cliente
ORDER BY cantidad ASC
LIMIT 5

,numero_de_cliente,cantidad
0,38592489,1
1,38803685,1
2,39769836,1
3,41250447,1
4,41467040,1


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



**Paso 1:** Castear las variables a los tipos correctos

In [8]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS
    SELECT
        *,
        (SUBSTR(foto_mes, 1, 4) || '-' || SUBSTR(foto_mes, 5, 2) || '-01')::DATE AS foto_mes_date
FROM competencia_01;

ALTER TABLE competencia_01 DROP COLUMN foto_mes;

ALTER TABLE competencia_01 RENAME COLUMN foto_mes_date TO foto_mes;

,Success


**Paso 2:** Calcular la fecha máxima y mínima de cada usuario, para rellenar los faltantes en medio (si los hay)

In [9]:
%%sql
SELECT
    DISTINCT numero_de_cliente,
    MIN(foto_mes) AS min_foto_mes,
    MAX(foto_mes) AS max_foto_mes
FROM competencia_01
GROUP BY numero_de_cliente
LIMIT 5;

,numero_de_cliente,min_foto_mes,max_foto_mes
0,125623682,2021-03-01,2021-05-01
1,133904099,2021-03-01,2021-05-01
2,137948526,2021-03-01,2021-05-01
3,125322827,2021-03-01,2021-05-01
4,127114037,2021-03-01,2021-05-01


**Paso 2:** 

In [10]:
%%sql
WITH cte AS (
SELECT
    numero_de_cliente,
    MAX(foto_mes) AS max_foto_mes,
    MIN(foto_mes) AS min_foto_mes,
    DATE_DIFF('month', MIN(foto_mes), MAX(foto_mes)) + 1 cantidad_fotos,
    COUNT(*) AS real_fotos
FROM competencia_01
GROUP BY numero_de_cliente
ORDER BY cantidad_fotos DESC)
SELECT
    numero_de_cliente,
    max_foto_mes,
    min_foto_mes,
    cantidad_fotos,
    real_fotos
FROM cte
WHERE cantidad_fotos <> real_fotos;

,numero_de_cliente,max_foto_mes,min_foto_mes,cantidad_fotos,real_fotos
0,95380426,2021-05-01,2021-03-01,3,2


In [11]:
%%sql
SELECT
    *
FROM competencia_01
WHERE numero_de_cliente = 95380426

,numero_de_cliente,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,foto_mes
0,95380426,1,0,0,52,153,-316.31,7892.03,1186.27,-3847.29,...,1,0.0,-23836.69,0.0,4574,6218.31,5,0,2744.82,2021-03-01
1,95380426,1,0,0,53,155,2535.14,6710.08,1023.54,-1229.32,...,5,0.0,-22629.28,0.0,4635,6500.14,3,0,5442.72,2021-05-01


In [12]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS (
    WITH RECURSIVE clientes_min_max_foto_mes AS (
    SELECT
        numero_de_cliente,
        MAX(foto_mes) AS max_foto_mes,
        MIN(foto_mes) AS min_foto_mes,
        DATE_DIFF('month', MIN(foto_mes), MAX(foto_mes)) + 1 cantidad_fotos,
        COUNT(*) AS real_fotos
    FROM competencia_01
    GROUP BY numero_de_cliente
    ORDER BY cantidad_fotos DESC
    ),
    ccf AS ( --ccf: clientes_con_faltantes
    SELECT
        numero_de_cliente,
        max_foto_mes,
        min_foto_mes
    FROM clientes_min_max_foto_mes
    WHERE cantidad_fotos <> real_fotos
    ),
    FotoMesFaltantes AS (
    SELECT
        ccf.numero_de_cliente,
        min_foto_mes AS foto_mes_sin_faltante
    FROM ccf
    UNION ALL
    SELECT
        FotoMesFaltantes.numero_de_cliente,
        foto_mes_sin_faltante + INTERVAL 1 MONTH
    FROM FotoMesFaltantes
    JOIN
        ccf ON foto_mes_sin_faltante + INTERVAL 1 MONTH <= ccf.max_foto_mes
    )
    SELECT
        c01.*,
        COALESCE(fmf.numero_de_cliente, c01.numero_de_cliente) AS numero_de_cliente_new,
        COALESCE(fmf.foto_mes_sin_faltante, c01.foto_mes) AS foto_mes_new
    FROM FotoMesFaltantes AS fmf
    FULL OUTER JOIN competencia_01 AS c01
    ON fmf.numero_de_cliente = c01.numero_de_cliente AND fmf.foto_mes_sin_faltante = c01.foto_mes
    ORDER BY numero_de_cliente_new, foto_mes_new ASC
    )
;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,491064


In [13]:
%%sql
SELECT
    COUNT(*) - COUNT(numero_de_cliente) AS cantidad_faltantes_cliente,
    COUNT(*) - COUNT(foto_mes) AS cantidad_faltantes_foto_mes,

    COUNT(*) - COUNT(numero_de_cliente_new) AS cantidad_faltantes_cliente_new,
    COUNT(*) - COUNT(foto_mes_new) AS cantidad_faltantes_foto_mes_new
FROM competencia_01
LIMIT 100

,cantidad_faltantes_cliente,cantidad_faltantes_foto_mes,cantidad_faltantes_cliente_new,cantidad_faltantes_foto_mes_new
0,1,1,0,0


In [14]:
%%sql
SELECT
    numero_de_cliente,
    foto_mes,
    numero_de_cliente_new,
    foto_mes_new
FROM competencia_01
WHERE numero_de_cliente_new = 95380426

,numero_de_cliente,foto_mes,numero_de_cliente_new,foto_mes_new
0,95380426.0,2021-03-01,95380426,2021-03-01
1,NaN,NaT,95380426,2021-04-01
2,95380426.0,2021-05-01,95380426,2021-05-01


In [15]:
%%sql
ALTER TABLE competencia_01
DROP COLUMN numero_de_cliente;

ALTER TABLE competencia_01
RENAME COLUMN numero_de_cliente_new TO numero_de_cliente;

ALTER TABLE competencia_01
DROP COLUMN foto_mes;

ALTER TABLE competencia_01
RENAME COLUMN foto_mes_new TO foto_mes;

,Success


In [16]:
%%sql
SELECT
    numero_de_cliente,
    foto_mes
FROM competencia_01
WHERE numero_de_cliente = 95380426

,numero_de_cliente,foto_mes
0,95380426,2021-03-01
1,95380426,2021-04-01
2,95380426,2021-05-01


In [17]:
%%sql
SELECT
    foto_mes
    , COUNT(*) AS cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
FROM competencia_01
GROUP BY foto_mes

,foto_mes,cantidad
0,2021-03-01,163324
1,2021-04-01,163638
2,2021-05-01,164102


In [18]:
%%sql
SELECT
    numero_de_cliente,
    foto_mes
FROM competencia_01
WHERE numero_de_cliente IN (95380426, 29183981, 29199353);

,numero_de_cliente,foto_mes
0,29183981,2021-03-01
1,29183981,2021-04-01
2,29183981,2021-05-01
3,29199353,2021-05-01
4,95380426,2021-03-01
5,95380426,2021-04-01
6,95380426,2021-05-01


In [19]:
%%sql
SELECT
    MIN(foto_mes) AS min_foto_mes,
    MAX(foto_mes) AS max_foto_mes
FROM competencia_01

,min_foto_mes,max_foto_mes
0,2021-03-01,2021-05-01


En el caso anterior, se esperan los siguientes resultaos:

|numero_de_cliente|foto_mes|clase_ternaria|
|-----------------|--------|--------------|
|29183981|2021-03-01|CONTINUA|
|29183981|2021-04-01|CONTINUA|
|29183981|2021-05-01|CONTINUA|
|29199353|2021-05-01|BAJA+1|
|95380426|2021-03-01|CONTINUA|
|95380426|2021-04-01|CONTINUA|
|95380426|2021-05-01|CONTINUA|


In [20]:
%%sql
SELECT 'HELLO'
--CREATE OR REPLACE TABLE competencia_01 AS (
--    SELECT
--        C01.*,
--        CASE
--            WHEN (C01.foto_mes != ('2021-03-01')::DATE) THEN NULL
--            WHEN (C01_1.foto_mes IS NULL) THEN 'BAJA+1'
--            WHEN (C01_2.foto_mes IS NULL) THEN 'BAJA+2'
--            ELSE 'CONTINUA'
--        END AS clase_ternaria
--    FROM competencia_01 C01
--    LEFT JOIN competencia_01 C01_1
--    ON (C01.numero_de_cliente = C01_1.numero_de_cliente)
--    AND (C01.foto_mes + INTERVAL 1 MONTH = C01_1.foto_mes)
--    LEFT JOIN competencia_01 C01_2
--    ON (C01.numero_de_cliente = C01_2.numero_de_cliente)
--    AND (C01.foto_mes + INTERVAL 2 MONTH = C01_2.foto_mes)
--)
--;

,'HELLO'
0,HELLO


In [21]:
%%sql
create or replace table competencia_01 as 
    with clientes as (
        select 
            numero_de_cliente,
            foto_mes,
            last_day(foto_mes + INTERVAL 1 MONTH) as mes_mas_1,
            last_day(foto_mes+ INTERVAL 2 MONTH) as mes_mas_2,
        from competencia_01
    ),

    mes_1 as (
        select 
            c.numero_de_cliente,
            c.foto_mes,
            c1.numero_de_cliente as cliente_mas_1
        from clientes c 
        left join competencia_01 c1
            on c.numero_de_cliente = c1.numero_de_cliente
            and c.mes_mas_1 = last_day(c1.foto_mes)
     ),

    mes_2 as (
        select 
            c.numero_de_cliente,
            c.foto_mes,
            c1.numero_de_cliente as cliente_mas_2
        from clientes c 
        left join competencia_01 c1
            on c.numero_de_cliente = c1.numero_de_cliente
            and c.mes_mas_2 = last_day( c1.foto_mes)
    ),

    max_mes as (
        select 
            MAX(foto_mes) as max_foto
        from competencia_01
        --group by 1
    ),

    final as (
    select 
        c1.*,      
        case
            when 
                cliente_mas_1 is not null 
                and cliente_mas_2 is null
                and c1.foto_mes = m.max_foto - INTERVAL 2 MONTH
            then 'BAJA+2'
            when 
                cliente_mas_1 is null 
                and cliente_mas_2 is null
                and c1.foto_mes <= m.max_foto - INTERVAL 1 MONTH
            then 'BAJA+1'
            when 
                (c1.foto_mes <= m.max_foto - INTERVAL 1 MONTH and cliente_mas_1 is not null)
                or 
                cliente_mas_2 is not null 
            then 'CONTINUA'
            else null end clase_ternaria
    from competencia_01 c1
    left join mes_1 m1 
        on m1.cliente_mas_1 = c1.numero_de_cliente
        and c1.foto_mes = m1.foto_mes
    left join mes_2 m2 
        on m2.cliente_mas_2 = c1.numero_de_cliente
        and c1.foto_mes = m2.foto_mes
    cross join max_mes m
    )
    select
       *
    from final

,Count
0,491064


In [22]:
%%sql
SELECT
    numero_de_cliente,
    foto_mes,
    clase_ternaria
FROM competencia_01_ternaria
WHERE numero_de_cliente IN (95380426, 29183981, 29199353)

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.CatalogException) Catalog Error: Table with name competencia_01_ternaria does not exist!
Did you mean "competencia_01"?
LINE 5: FROM competencia_01_ternaria
             ^
[SQL: SELECT
    numero_de_cliente,
    foto_mes,
    clase_ternaria
FROM competencia_01_ternaria
WHERE numero_de_cliente IN (95380426, 29183981, 29199353)]
(Background on this error at: https://sqlalche.me/e/14/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


In [23]:
%%sql
CREATE OR REPLACE TABLE competencia_01 AS (
    SELECT
        c.*,
        STRFTIME(foto_mes, '%Y%m')::UINTEGER AS foto_mes_new
    FROM competencia_01 AS c
);

,Count
0,491064


In [24]:
%%sql
ALTER TABLE competencia_01
DROP COLUMN foto_mes;

ALTER TABLE competencia_01
RENAME COLUMN foto_mes_new TO foto_mes;

,Success


In [25]:
%%sql
SELECT
    *
FROM competencia_01
LIMIT 5;

,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,...,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,numero_de_cliente,clase_ternaria,foto_mes
0,1,0,1,55,58,5591.03,6186.97,389.47,3805.34,629.07,...,-16422.00,0.0,1661,0.00,0,0,14122.92,85761200,CONTINUA,202103
1,1,0,0,60,165,-289.29,4084.69,188.64,-1017.58,610.13,...,0.00,0.0,5048,16961.08,6,0,3366.51,85762140,CONTINUA,202103
2,1,0,0,51,165,-8545.91,-71486.16,3310.95,-11135.87,1027.71,...,-54544.50,0.0,3969,12375.24,14,0,8985.18,85764924,CONTINUA,202103
3,1,0,0,61,135,2877.25,30532.97,2914.51,-1687.24,1655.47,...,-55096.09,0.0,4004,39194.17,19,0,2580.60,85765137,CONTINUA,202103
4,1,0,0,39,165,-3578.77,-24644.51,546.03,-3836.88,320.42,...,-62051.70,0.0,5048,18440.19,7,0,8328.30,85768041,CONTINUA,202103


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [26]:
%%sql
select
    clase_ternaria,
    count(*) as cant,
    cant/(SELECT count(*) from competencia_01) as prop_tot,
    cant/(SELECT count(*) from competencia_01 WHERE clase_ternaria is not NULL) as prop_m0
from competencia_01
where foto_mes = '2021-03-01'
group by clase_ternaria

,clase_ternaria,cant,prop_tot,prop_m0


In [36]:
%%sql
SELECT
    *
FROM competencia_01
LIMIT 5;

,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,...,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,numero_de_cliente,clase_ternaria,foto_mes
0,1,0,1,55,58,5591.03,6186.97,389.47,3805.34,629.07,...,-16422.00,0.0,1661,0.00,0,0,14122.92,85761200,CONTINUA,202103
1,1,0,0,60,165,-289.29,4084.69,188.64,-1017.58,610.13,...,0.00,0.0,5048,16961.08,6,0,3366.51,85762140,CONTINUA,202103
2,1,0,0,51,165,-8545.91,-71486.16,3310.95,-11135.87,1027.71,...,-54544.50,0.0,3969,12375.24,14,0,8985.18,85764924,CONTINUA,202103
3,1,0,0,61,135,2877.25,30532.97,2914.51,-1687.24,1655.47,...,-55096.09,0.0,4004,39194.17,19,0,2580.60,85765137,CONTINUA,202103
4,1,0,0,39,165,-3578.77,-24644.51,546.03,-3836.88,320.42,...,-62051.70,0.0,5048,18440.19,7,0,8328.30,85768041,CONTINUA,202103


Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [27]:
%%sql
COPY competencia_01 TO '../../datasets/interim/competencia_01.csv' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,491064


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo

In [28]:
%%sql
CREATE OR REPLACE TABLE ejercicio_target AS
SELECT
    *
FROM read_csv_auto('../../datasets/raw/ejercicio_target.csv')

,Count
0,739


In [29]:
%%sql
CREATE OR REPLACE TABLE ejercicio_target AS
SELECT
    nombre,
    periodo,
    (SUBSTR(periodo, 1, 4) || '-' || SUBSTR(periodo, 5, 2) || '-01')::DATE AS foto_mes
FROM ejercicio_target

,Count
0,739


In [30]:
%%sql
WITH min_max_foto_mes AS (
SELECT
    nombre,
    MAX(foto_mes) AS max_foto_mes,
    MIN(foto_mes) AS min_foto_mes,
    DATE_DIFF('month', MIN(foto_mes), MAX(foto_mes)) + 1 cantidad_fotos,
    COUNT(*) AS real_fotos
FROM ejercicio_target
GROUP BY nombre
ORDER BY cantidad_fotos DESC)
SELECT
    nombre,
    max_foto_mes,
    min_foto_mes,
    cantidad_fotos,
    real_fotos
FROM min_max_foto_mes
WHERE cantidad_fotos <> real_fotos;

,nombre,max_foto_mes,min_foto_mes,cantidad_fotos,real_fotos
0,Rebeca Toledo Tercero,2022-03-01,2021-10-01,6,5


In [31]:
%%sql
CREATE OR REPLACE TABLE ejercicio_target AS (
    WITH RECURSIVE clientes_min_max_foto_mes AS (
    SELECT
        nombre,
        MAX(foto_mes) AS max_foto_mes,
        MIN(foto_mes) AS min_foto_mes,
        DATE_DIFF('month', MIN(foto_mes), MAX(foto_mes)) + 1 cantidad_fotos,
        COUNT(*) AS real_fotos
    FROM ejercicio_target
    GROUP BY nombre
    ORDER BY cantidad_fotos DESC
    ),
    ccf AS ( --ccf: clientes_con_faltantes
    SELECT
        nombre,
        max_foto_mes,
        min_foto_mes
    FROM clientes_min_max_foto_mes
    WHERE cantidad_fotos <> real_fotos
    ),
    FotoMesFaltantes AS (
    SELECT
        ccf.nombre,
        min_foto_mes AS foto_mes_sin_faltante
    FROM ccf
    UNION ALL
    SELECT
        FotoMesFaltantes.nombre,
        foto_mes_sin_faltante + INTERVAL 1 MONTH
    FROM FotoMesFaltantes
    JOIN
        ccf ON foto_mes_sin_faltante + INTERVAL 1 MONTH <= ccf.max_foto_mes
    )
    SELECT
        c01.*,
        COALESCE(fmf.nombre, c01.nombre) AS nombre_new,
        COALESCE(fmf.foto_mes_sin_faltante, c01.foto_mes) AS foto_mes_new
    FROM FotoMesFaltantes AS fmf
    FULL OUTER JOIN ejercicio_target AS c01
    ON fmf.nombre = c01.nombre AND fmf.foto_mes_sin_faltante = c01.foto_mes
    ORDER BY nombre_new, foto_mes_new ASC
    )
;

,Count
0,740


In [32]:
%%sql
ALTER TABLE ejercicio_target
DROP COLUMN nombre;

ALTER TABLE ejercicio_target
DROP COLUMN periodo;

ALTER TABLE ejercicio_target
DROP COLUMN foto_mes;

ALTER TABLE ejercicio_target
RENAME COLUMN nombre_new TO nombre;

ALTER TABLE ejercicio_target
RENAME COLUMN foto_mes_new TO foto_mes;

,Success


In [33]:
%%sql
SELECT
    *
FROM ejercicio_target
WHERE nombre = 'Rebeca Toledo Tercero'
LIMIT 5;

,nombre,foto_mes
0,Rebeca Toledo Tercero,2021-10-01
1,Rebeca Toledo Tercero,2021-11-01
2,Rebeca Toledo Tercero,2021-12-01
3,Rebeca Toledo Tercero,2022-01-01
4,Rebeca Toledo Tercero,2022-02-01


In [34]:
%%sql
CREATE OR REPLACE TABLE ejercicio_target AS (
    SELECT
        *,
        'BAJA+2' AS clase_ternaria
    FROM ejercicio_target
)

,Count
0,740


In [35]:
%%sql
COPY ejercicio_target TO '../../datasets/interim/ejercicio_target.csv' (FORMAT CSV, HEADER)

,Count
0,740
